AI Content Assistant using OpenAI API (via HTTP Requests)
---------------------------------------------------------

📌 Description:
This script interacts with the OpenAI Chat Completions endpoint to generate
content such as emails, captions, or short documentation.

Features:
- Uses the 'requests' library for API calls
- Handles network and API errors with exponential backoff retry strategy
- Validates user input
- Secure API key management via environment variable
- Logs key operations for better debugging and optimization

⚙️ Setup Instructions:

1 Install dependencies:
    pip install requests

2 Set your OpenAI API key:
    export OPENAI_API_KEY="your_api_key_here"

In [ ]:


import os
import time
import logging
import requests

# Read API key securely from environment variable
API_KEY = os.getenv("OPENAI_API_KEY")
if not API_KEY:
    raise ValueError("Missing API Key! Set the OPENAI_API_KEY environment variable.")

URL = "https://api.openai.com/v1/chat/completions"

# Use standard header format for OpenAI API
HEADERS = {
    "Authorization": f"Bearer {API_KEY}",
    "Content-Type": "application/json"
}

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s"
)

# ---------- CORE FUNCTION ------#
def get_ai_response(user_prompt: str, retries: int = 3, timeout: int = 20):
    """
    Send a user prompt to the OpenAI Chat API and return the model's response.
    """
    if not user_prompt or not user_prompt.strip():
        return "⚠️ Please enter a valid prompt. Empty input not allowed."

    # Prepare API request payload
    message = {
        "model": "gpt-4",
        "messages": [
            {
                "role": "system",
                "content": (
                    "You are a professional content writer who writes concise, clear "
                    "emails, captions, and documentation under 100 tokens."
                )
            }, {"role": "user", "content": f"Write for the following request: {user_prompt}"}
        ],
        "temperature": 0.9, "max_tokens": 100
    }

    for attempt in range(retries):
        try:
            logging.info(f"Attempt {attempt + 1}: Sending request to OpenAI API...")
            response = requests.post(URL, headers=HEADERS, json=message, timeout=timeout)

            # Handle HTTP errors gracefully
            if response.status_code != 200:
                logging.error(f"API Error {response.status_code}: {response.text}")
                raise requests.exceptions.RequestException(response.text)

            # Extract model's response safely
            data = response.json()
            return data["choices"][0]["message"]["content"].strip()

        except requests.exceptions.Timeout:
            logging.warning("Request timed out. Retrying...")
        except requests.exceptions.ConnectionError:
            logging.warning("Network error. Check your connection.")
        except requests.exceptions.RequestException as e:
            logging.error(f"API Request failed: {e}")
        except KeyError:
            logging.error("Unexpected response format received.")
        except Exception as e:
            logging.error(f"Unexpected error: {e}")

        wait_time = 2 ** attempt
        logging.info(f"Retrying in {wait_time} seconds...")
        time.sleep(wait_time)

    return "❌ Failed to get a response after multiple attempts. Please try again later."


#Main Function
def main():
    """Main entry point for user input and output handling."""
    logging.info("🚀 AI Content Assistant Started.")
    try:
        user_input = input("Enter your prompt: ").strip()
        response = get_ai_response(user_input)
        print("\nAI Response:\n", response)
    except KeyboardInterrupt:
        print("\nExiting gracefully. Goodbye!")
    except Exception as e:
        logging.error(f"Critical error: {e}")
        print("❌ An unexpected error occurred. Please try again later.")


if __name__ == "__main__":
    main()
